In [2]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

# 2.16.1 Statistical functions

In [5]:
ser = pd.Series(np.random.randn(8))
ser.pct_change()

0         NaN
1   -1.923373
2   -2.931038
3   -0.935503
4   -0.123427
5    7.136023
6   -1.639340
7   -2.450775
dtype: float64

In [7]:
df = DataFrame(np.random.rand(10, 4))
df.pct_change(3)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,-0.020591,12.475995,0.967325,-0.553468
4,1.522533,-0.759493,4.268283,-0.128637
5,-0.839266,4.969317,-0.877771,-0.342039
6,-0.230832,0.994627,-0.320743,1.306429
7,0.511824,3.978803,2.442230,-0.767423
8,1.038695,14.972710,17.444836,0.235789
9,-0.085083,0.054706,0.009563,-0.911786


In [14]:
s = pd.Series(range(1,6))
print(s)
s.pct_change(3)

0    1
1    2
2    3
3    4
4    5
dtype: int64


0    NaN
1    NaN
2    NaN
3    3.0
4    1.5
dtype: float64

In [16]:
s1 = pd.Series(range(1,11))
s2 = pd.Series(range(2,21,2))
s1.cov(s2)

18.333333333333332

In [18]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=["a", "b", "c", "d", "e"])
frame.cov()

,a,b,c,d,e
a,1.050728,0.021590,-0.033379,0.031233,0.035940
b,0.021590,1.030669,0.022360,-0.035406,0.055048
c,-0.033379,0.022360,1.059442,0.056878,-0.038169
d,0.031233,-0.035406,0.056878,0.933382,0.052404
e,0.035940,0.055048,-0.038169,0.052404,0.895523


In [20]:
frame = pd.DataFrame(np.random.randn(20, 3), columns=["a", "b", "c"])
frame.loc[frame.index[:5], "a"] = np.nan
frame.loc[frame.index[5:10], "b"] = np.nan
frame.cov()
frame.cov(min_periods=12)

,a,b,c
a,0.969831,NaN,0.318122
b,NaN,0.937038,-0.037230
c,0.318122,-0.037230,0.880216


In [24]:
s1 = pd.Series(range(1,6))
s2 = pd.Series(range(6,11))
s2[:3] = np.nan
s1.cov(s2, min_periods=2)

0.5

In [26]:
frame = pd.DataFrame(np.random.randn(20, 3), columns=["a", "b", "c"])
frame.loc[frame.index[:5], "a"] = np.nan
frame.loc[frame.index[5:10], "b"] = np.nan
frame.corr()
frame.corr(min_periods=12)

,a,b,c
a,1.000000,NaN,-0.268127
b,NaN,1.000000,-0.058104
c,-0.268127,-0.058104,1.000000


In [27]:
def histogram_intersection(a, b):
    return np.minimum(np.true_divide(a, a.sum()), 
                      np.true_divide(b, b.sum())).sum()

frame.corr(method=histogram_intersection)

,a,b,c
a,1.000000,-0.449048,-6.072194
b,-0.449048,1.000000,-0.541214
c,-6.072194,-0.541214,1.000000


In [28]:
index = ["a", "b", "c", "d", "e"]
columns = ["one", "two", "three", "four"]
df1 = pd.DataFrame(np.random.randn(5, 4), index=index, columns=columns)
df2 = pd.DataFrame(np.random.randn(4, 4), index=index[:4], columns=columns)

df1.corrwith(df2)

one      0.517229
two     -0.014314
three   -0.362604
four    -0.563880
dtype: float64

In [30]:
s = pd.Series(np.random.randn(5), index=list("abcde"))
s["d"] = s["b"] # so there's a tie
s.rank()

a    1.688946
b    1.489742
c   -0.845329
d    1.489742
e    0.695434
dtype: float64

In [31]:
s.rank()

a    5.0
b    3.5
c    1.0
d    3.5
e    2.0
dtype: float64

In [36]:
df = pd.DataFrame(np.random.randn(10, 6))
df[4] = df[2][:5] # some ties
df.rank(1, ascending=False)

,0,1,2,3,4,5
0,6.0,1.0,2.5,4.0,2.5,5.0
1,6.0,3.0,1.5,4.0,1.5,5.0
2,5.0,4.0,2.5,1.0,2.5,6.0
3,3.0,1.0,4.5,2.0,4.5,6.0
4,2.0,4.0,5.5,1.0,5.5,3.0
5,3.0,1.0,4.0,5.0,NaN,2.0
6,4.0,1.0,3.0,5.0,NaN,2.0
7,3.0,1.0,4.0,2.0,NaN,5.0
8,5.0,4.0,3.0,2.0,NaN,1.0
9,4.0,2.0,3.0,1.0,NaN,5.0
